In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import utils

# SKLearn related imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from lightgbm import LGBMRegressor  # or any other estimator

plt.rcParams['figure.figsize'] = (12, 4.2)

In [ ]:
### dataset 1
df1 = pd.read_csv("train/chain_campaigns.csv")
# Convert dates and compute duration
df1['start_date'] = pd.to_datetime(df1['start_date'])
df1['end_date'] = pd.to_datetime(df1['end_date'])
df1['duration_days'] = (df1['end_date'] - df1['start_date']).dt.days + 1
df1.head()

In [ ]:
### dataset 2
df2 = pd.read_csv("train/product_prices_leaflets.csv")
df2['time_key'] = pd.to_datetime(df2['time_key'].astype(str), format='%Y%m%d') #convert time_key to datetime
#df2 = df2.set_index(['sku','time_key']).sort_index()

#drop negative discount values
neg_count = (df2['discount'] < 0).sum()
print(f"Dropping {neg_count} rows with negative discount")

# Filter them out
df2 = df2[df2['discount'] >= 0].copy()

df2['effective_price'] = df2['pvp_was'] * (1 - df2['discount'])

df2.head()

In [ ]:
df2 = df2.drop(columns=['leaflet']) #remove "leaflet" column for now

In [ ]:
df2.head()

In [ ]:
duplicates = df2.duplicated().sum()
print(f"Exact duplicate rows: {duplicates}")

In [ ]:
#### remove duplicates from df2
df2_cleaned = df2.drop_duplicates()

In [ ]:
###dataset 3
df3 = pd.read_csv("train/product_structures_sales.csv")

df3['time_key'] = pd.to_datetime(df3['time_key'].astype(str), format='%Y%m%d') #convert time_key to datetime
#df3 = df3.set_index(['time_key','sku']).sort_index()

#drop negative quantity values
neg_count = (df3['quantity'] < 0).sum()
print(f"Dropping {neg_count} rows with negative quantity")

# Filter them out
df3 = df3[df3['quantity'] >= 0].copy()

#transform structure_levels into categorical variables
structure_cols = [
    'structure_level_1',
    'structure_level_2',
    'structure_level_3',
    'structure_level_4'
]

for col in structure_cols:
    df3[col] = df3[col].astype('category')

df3.head()

In [ ]:
#### Only get the competitor A and chain prices (eliminate competitor B from df2)
df2_comp_A = df2_cleaned[df2_cleaned['competitor'] != 'competitorB'].copy()

In [ ]:
df_merged = pd.merge(
    df2_comp_A,
    df3,
    on=['sku','time_key'],
    how='left' #keep all df2 skus
)

df_merged.head()

In [ ]:
duplicates_merged = df_merged.duplicated().sum()
print(f"Exact duplicate rows: {duplicates_merged}")

In [ ]:
df2['competitor'].value_counts()

In [ ]:
### separate data chain vs competitor A

# 1. Split out the two series
df_chain = df_merged[df_merged['competitor'] == 'chain'].copy()
df_A = df_merged[df_merged['competitor'] == 'competitorA'].copy()

# 2. Drop the old competitor column and rename the remaining columns
df_chain = df_chain.drop(columns='competitor').rename(columns={
    'pvp_was':      'pvp_was_chain',
    'discount':     'discount_chain',
    'flag_promo':   'flag_promo_chain',
    'quantity':   'quantity_chain',
    'effective_price': 'effective_price_chain'  # chain doesn't have effective_price target
})

df_A = df_A.drop(columns='competitor').rename(columns={
    'pvp_was':        'pvp_was_A',
    'discount':       'discount_A',
    'flag_promo':     'flag_promo_A',
    'quantity':   'quantity_A',
    'effective_price': 'effective_price_A'
})

# 3. Merge back together on sku & time_key
df_wide_prices = pd.merge(
    df_chain,
    df_A,
    on=['sku','time_key'],
    how='inner',  # or 'outer' if you want to keep rows missing one side
)

In [ ]:
df_wide_prices.shape ## ver se os valores desta nova dataframe fazem sentido --> fiquei aqui; se sim, desenvolver modelo

In [ ]:
df_wide_prices.head()

In [ ]:
df = df_wide_prices.drop(columns=['effective_price_chain', 'discount_A', 'quantity_A', 'structure_level_4_y', 'structure_level_3_y', 'structure_level_2_y', 'structure_level_1_y'])

In [ ]:
df = df.rename(columns={
    'structure_level_1_x': 'structure_level_1',
    'structure_level_2_x': 'structure_level_2',
    'structure_level_3_x': 'structure_level_3',
    'structure_level_4_x': 'structure_level_4'
})


In [ ]:
df.head()

In [ ]:
df['month'] = df['time_key'].dt.month        
df['day_of_week'] = df['time_key'].dt.weekday      
df['day_of_month']= df['time_key'].dt.day         
df['year'] = df['time_key'].dt.year
df['is_weekend']  = (df['time_key'].dt.weekday >= 5).astype(int)

In [ ]:
### Impute via augmented masking

df = df.sort_values(['sku','time_key']).reset_index(drop=True)

# 2) Compute lag features on pvp_was_A
df['price_lag_1_A'] = df.groupby('sku')['pvp_was_A'].shift(1)
df['price_roll_mean_7_A'] = (
    df.groupby('sku')['pvp_was_A']
      .shift(1)
      .rolling(7)
      .mean()
      .reset_index(level=0, drop=True)
)

# 3) Build your imputation rule: category‐mean spread
df_warm = df.dropna(subset=['price_lag_1_A'])
df_warm['spread'] = df_warm['price_lag_1_A'] - df_warm['pvp_was_chain']
cat_spread   = df_warm.groupby('structure_level_2')['spread'].mean().to_dict()
global_spread = df_warm['spread'].mean()

def impute_lags(row):
    sp   = cat_spread.get(row['structure_level_2'], global_spread)
    base = row['pvp_was_chain'] + sp
    return base, base

# 4) Create the augmentation mask on 10% of warm rows
rng      = np.random.default_rng(42)
warm_idx = df[df['price_lag_1_A'].notna()].index
mask_idx = rng.choice(warm_idx, size=int(len(warm_idx)*0.1), replace=False)

# 5) Initialize the flag
df['is_imputed_A'] = 0

# 6) Apply masking: overwrite lag features & set flag
df.loc[mask_idx, 'is_imputed_A'] = 1
for idx in mask_idx:
    lag1, roll7 = impute_lags(df.loc[idx])
    df.at[idx, 'price_lag_1_A']       = lag1
    df.at[idx, 'price_roll_mean_7_A'] = roll7

# Now:
#  - `price_lag_1_A` and `price_roll_mean_7_A` are real for 90% of warm SKUs, imputed for 10%
#  - `is_imputed_A` = 1 marks the imputed cases
#  - cold-start SKUs (NaN lags) remain NaN here and you’ll impute them at inference

# 7) Ready for time‐split and model training

feature_cols = [
    'sku', 'pvp_was_chain', 'discount_chain',
    'flag_promo_chain', 'structure_level_4', 'structure_level_3',
        'structure_level_2', 'structure_level_1', 'quantity_chain', 'flag_promo_A', 'month', 'day_of_week', 'day_of_month', 'year', 'price_lag_1_A','price_roll_mean_7_A']

X = df[feature_cols]
y = df['effective_price_A']

In [ ]:
#feature_cols = [
#    'sku', 'pvp_was_chain', 'discount_chain',
#    'flag_promo_chain', 'structure_level_4', 'structure_level_3',
#    'structure_level_2', 'structure_level_1', 'quantity_chain', 'flag_promo_A', 'month', 'day_of_week', 'day_of_month', 'year']

#X = df[feature_cols]
#y = df['effective_price_A']

In [ ]:
### train test split time-based & split into X and y (train and test sets)
split = int(len(df) * 0.8)

train_X, test_X = X.iloc[:split], X.iloc[split:]
train_y, test_y = y.iloc[:split], y.iloc[split:]

print(f"Train from {df['time_key'].iloc[0].date()} to {df['time_key'].iloc[split-1].date()}")
print(f"Test  from {df['time_key'].iloc[split].date()} to {df['time_key'].iloc[-1].date()}")

In [ ]:
categorical_cols = ['sku', 'structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4', 'flag_promo_chain', 'flag_promo_A', 'month', 'day_of_week']
numeric_cols     = ['pvp_was_chain', 'discount_chain', 'quantity_chain', 'year', 'day_of_month', 'price_lag_1_A', 'price_roll_mean_7_A']

In [ ]:
df.head()

In [ ]:
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
    ("num", RobustScaler(), numeric_cols)
])

In [ ]:
### Model pipeline

def model_pipeline(train_X, test_X, train_y, test_y, n_estimators=100, random_state=42):

    clf = Pipeline([
        ('preproc',preprocessor),
        ('regressor', LGBMRegressor(n_estimators=n_estimators, random_state=random_state))])

    clf.fit(train_X, train_y)

    y_pred = clf.predict(test_X)
    
    mae = mean_absolute_error(test_y, y_pred)
    print(f"MAE: {mae}")
    
    mape = (abs(test_y - y_pred) / test_y).mean() * 100
    print(f"MAPE: {mape:.1f}%")

    return clf, y_pred, mae, mape

In [ ]:
### Model evaluation (MAE)
model, preds, mae, mape = model_pipeline(train_X, test_X, train_y, test_y)

In [ ]:
#serialize columns
import json
import pickle

with open('columns_A_1.json', 'w') as fh:
    json.dump(train_X.columns.tolist(), fh)
    

#serialize dtypes of the columns

with open('dtypes_A_1.pickle', 'wb') as fh:
    pickle.dump(train_X.dtypes, fh)
    
    
#serialize fitted pipeline
import joblib

joblib.dump(model, 'pipeline_A_1.pickle') 